In [1]:
import torch
import torch.nn as nn
from utils import train
from bin_packing_dataset import BinPackingDataset
from bin_packing_model import BinPackingLSTMModel
from torch.utils.data import random_split

In [2]:
# Fijamos la semilla para que los resultados sean reproducibles
SEED = 23

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# Algunas constantes

# definimos el dispositivo que vamos a usar
DEVICE = "cpu"  # por defecto, usamos la CPU
if torch.cuda.is_available():
    DEVICE = "cuda"  # si hay GPU, usamos la GPU
elif torch.backends.mps.is_available():
    DEVICE = "mps"  # si no hay GPU, pero hay MPS, usamos MPS

NUM_WORKERS = 0 # max(os.cpu_count() - 1, 1)  # número de workers para cargar los datos


print(f"Device: {DEVICE}")
print(f"Num Workers: {NUM_WORKERS}")

Device: mps
Num Workers: 0


### Exploración del Dataset

In [4]:
#Creacion del dataset de entrenamiento, validacion y test

full_dataset = BinPackingDataset('data2')
print('Full dataset size:', len(full_dataset))
container_tensor, boxes_tensor = full_dataset[0]
print('Container:', container_tensor)
print('Boxes:', boxes_tensor)

train_dataset, val_dataset = random_split(full_dataset, [int(0.8*len(full_dataset)), int(0.20*len(full_dataset))])
print('Train dataset size:', len(train_dataset))
print('Val dataset size:', len(val_dataset))

Full dataset size: 10000
Container: tensor([9., 6.])
Boxes: tensor([[2., 4.],
        [7., 4.],
        [2., 5.],
        [3., 1.],
        [4., 1.],
        [0., 0.]])
Train dataset size: 8000
Val dataset size: 2000


In [5]:
# Collate para manejar secuencias de diferentes longitudes
import torch.nn.utils.rnn as rnn_utils

def custom_collate_fn_with_padding(batch):
    """
    Collate function que mantiene la estructura de contenedor y agrega padding a las secuencias de cajas.
    
    Args:
        batch (list): Lista de tuplas (contenedor, cajas).
        
    Returns:
        tuple: (contenedores, cajas_padded, longitudes) donde:
            - contenedores: Tensor de tamaño (batch_size, 2).
            - cajas_padded: Tensor de tamaño (batch_size, max_len, 2) con padding.
            - longitudes: Tensor de tamaños originales de las secuencias de cajas.
    """
    containers = torch.stack([item[0] for item in batch])  # Contenedores como tensor
    boxes = [item[1] for item in batch]  # Lista de cajas
    
    # Padding de las secuencias de cajas (rellenar con ceros hasta la longitud máxima en el batch)
    boxes_padded = rnn_utils.pad_sequence(boxes, batch_first=True)
    
    # Longitudes originales de cada secuencia de cajas
    lengths = torch.tensor([len(b) for b in boxes])
    
    return containers, boxes_padded



BATCH_SIZE = 40000
mock_loader = torch.utils.data.DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn_with_padding)

container, target_seq = next(iter(mock_loader))
print('Tamaño del primer contenedor:', container.shape)
print('Tamaño de las cajas del primer contenedor:', target_seq.shape)


train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn_with_padding)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn_with_padding)

Tamaño del primer contenedor: torch.Size([10000, 2])
Tamaño de las cajas del primer contenedor: torch.Size([10000, 11, 2])


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AutoRegressiveBinPackingModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, max_dim, n_layers, dropout=0.1):
        super(AutoRegressiveBinPackingModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.max_dim = max_dim
        
        # Embedding para entrada (contenedor y cajas)
        self.embedding = nn.Linear(input_dim, hidden_dim)
        
        # LSTM para modelar secuencias
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout
        )
        
        # Proyección para obtener logits de distribuciones de width y height
        self.fc_width = nn.Linear(hidden_dim, max_dim + 1)  # +1 para incluir el token EOS
        self.fc_height = nn.Linear(hidden_dim, max_dim + 1)
    
    def forward(self, container, target_seq=None, seq_len=100, teacher_forcing_ratio=0.5):
        """
        Args:
            container: Tensor con las dimensiones del contenedor (batch_size, input_dim).
            target_seq: Secuencia objetivo durante entrenamiento (batch_size, seq_len, input_dim).
            seq_len: Longitud máxima de secuencia durante generación.
            teacher_forcing_ratio: Probabilidad de usar teacher forcing (0.0 a 1.0).
        
        Returns:
            Logits para distribuciones de width y height.
        """
        container_emb = self.embedding(container).unsqueeze(1)  # (batch_size, 1, hidden_dim)
        
        # Para almacenar las salidas durante la generación
        outputs_width = []
        outputs_height = []
        
        # Estado inicial
        generated_seq = container_emb
        hidden = None
        
        for t in range(seq_len):
            output, hidden = self.lstm(generated_seq, hidden)  # (batch_size, 1, hidden_dim)
            
            # Logits para width y height
            logits_width = self.fc_width(output[:, -1, :])  # (batch_size, max_dim+1)
            logits_height = self.fc_height(output[:, -1, :])  # (batch_size, max_dim+1)
            outputs_width.append(logits_width)
            outputs_height.append(logits_height)
            
            if target_seq is not None and torch.rand(1).item() < teacher_forcing_ratio:
                # Usar la secuencia objetivo (teacher forcing)
                next_box = target_seq[:, t, :]  # (batch_size, 2)
            else:
                # Sampleo de la predicción
                prob_width = F.softmax(logits_width, dim=-1)  # (batch_size, max_dim+1)
                prob_height = F.softmax(logits_height, dim=-1)  # (batch_size, max_dim+1)
                next_width = torch.multinomial(prob_width, num_samples=1)  # (batch_size, 1)
                next_height = torch.multinomial(prob_height, num_samples=1)  # (batch_size, 1)
                next_box = torch.cat([next_width, next_height], dim=1)  # (batch_size, 2)
            
            # Preparar la entrada para el siguiente paso
            next_box_emb = self.embedding(next_box.float())  # Convertir a embedding
            # Concatenamos la secuencia generada con la nueva caja
            generated_seq = torch.cat([generated_seq, next_box_emb.unsqueeze(1)], dim=1)
        
        # Apilar las salidas
        logits_width = torch.stack(outputs_width, dim=1)  # (batch_size, seq_len, max_dim+1)
        logits_height = torch.stack(outputs_height, dim=1)  # (batch_size, seq_len, max_dim+1)
        return logits_width, logits_height


In [19]:
# Configuración
from utils import EarlyStopping


MAX_DIM = 20
input_dim = 2
hidden_dim = 30
n_layers = 1
dropout = 0
epochs = 500
teacher_forcing_ratio = 1.0  # Iniciamos con teacher forcing completo
teacher_forcing_decay = 0.95  # Decae 5% por época

# Modelo
model = AutoRegressiveBinPackingModel(input_dim, hidden_dim, MAX_DIM, n_layers, dropout)
model.to(DEVICE)
# Hiperparámetros
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
teacher_forcing_ratio = 0.5  # 50% de probabilidad de usar teacher forcing

def evaluate(model, criterion, data_loader, device):
    """
    Evalúa el modelo en los datos proporcionados y calcula la pérdida promedio.

    Args:
        model (torch.nn.Module): El modelo que se va a evaluar.
        criterion (torch.nn.Module): La función de pérdida que se utilizará para calcular la pérdida.
        data_loader (torch.utils.data.DataLoader): DataLoader que proporciona los datos de evaluación.

    Returns:
        float: La pérdida promedio en el conjunto de datos de evaluación.

    """
    model.eval()  # ponemos el modelo en modo de evaluacion
    total_loss = 0  # acumulador de la perdida
    with torch.no_grad():  # deshabilitamos el calculo de gradientes
        for container, target_seq in data_loader:  # iteramos sobre el dataloader
            container = container.to(device)  # movemos los datos al dispositivo
            target_seq = target_seq.to(device)  # movemos los datos al dispositivo
            logits_width, logits_height = model(
                container, 
                target_seq=target_seq, 
                seq_len=target_seq.size(1), 
                teacher_forcing_ratio=teacher_forcing_ratio
            )
            target_width = target_seq[:, :, 0].long()  # (batch_size, seq_len)
            target_height = target_seq[:, :, 1].long()  # (batch_size, seq_len)
            loss_width = criterion(logits_width.view(-1, MAX_DIM+1), target_width.view(-1))
            loss_height = criterion(logits_height.view(-1, MAX_DIM+1), target_height.view(-1))
            loss = (loss_width + loss_height).item()
            total_loss += loss  # acumulamos la perdida
    return total_loss / len(data_loader)  # retornamos la perdida promedio

early_stopping = EarlyStopping(patience=7)

for epoch in range(epochs):
    model.train()
    teacher_forcing_ratio *= teacher_forcing_decay  # Reducimos el ratio por cada época
    train_loss = 0
    
    for container, target_seq in train_dataloader:
        container = container.to(DEVICE)
        target_seq = target_seq.to(DEVICE)
        
        optimizer.zero_grad()
        logits_width, logits_height = model(
            container, 
            target_seq=target_seq, 
            seq_len=target_seq.size(1), 
            teacher_forcing_ratio=teacher_forcing_ratio
        )
        
        # Aplanar las salidas para calcular la pérdida
        target_width = target_seq[:, :, 0].long()  # (batch_size, seq_len)
        target_height = target_seq[:, :, 1].long()  # (batch_size, seq_len)
        loss_width = criterion(logits_width.view(-1, MAX_DIM+1), target_width.view(-1))
        loss_height = criterion(logits_height.view(-1, MAX_DIM+1), target_height.view(-1))
        
        loss = loss_width + loss_height
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_dataloader)}")
    val_loss = evaluate(model, criterion, val_dataloader, DEVICE)
    print(f"Epoch {epoch+1}/{epochs}, Val Loss: {val_loss}")
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Detener entrenamiento en la época {epoch}, la mejor pérdida fue {early_stopping.best_score:.5f}")
        break





Epoch 1/500, Train Loss: 6.229360580444336
Epoch 1/500, Val Loss: 6.197573184967041
Epoch 2/500, Train Loss: 6.174419403076172
Epoch 2/500, Val Loss: 6.142951965332031
Epoch 3/500, Train Loss: 6.13849401473999
Epoch 3/500, Val Loss: 6.09564208984375
Epoch 4/500, Train Loss: 6.109881401062012
Epoch 4/500, Val Loss: 6.032200813293457
Epoch 5/500, Train Loss: 6.045889854431152
Epoch 5/500, Val Loss: 5.986453056335449
Epoch 6/500, Train Loss: 5.998867511749268
Epoch 6/500, Val Loss: 5.943309783935547
Epoch 7/500, Train Loss: 5.9423441886901855
Epoch 7/500, Val Loss: 5.890051364898682
Epoch 8/500, Train Loss: 5.892148971557617
Epoch 8/500, Val Loss: 5.862393379211426
Epoch 9/500, Train Loss: 5.832493305206299
Epoch 9/500, Val Loss: 5.788456916809082
Epoch 10/500, Train Loss: 5.805131912231445
Epoch 10/500, Val Loss: 5.752199172973633
Epoch 11/500, Train Loss: 5.7267560958862305
Epoch 11/500, Val Loss: 5.705105781555176
Epoch 12/500, Train Loss: 5.667925834655762
Epoch 12/500, Val Loss: 5.62

In [20]:
from models import BinPackingGame, Box, ResolvedBinPackingGameResult
dataset_keys = set()

def tensor_to_box(tensor):
    return Box(int(tensor[0].item()), int(tensor[1].item()))
    

for container_tensor, boxes_tensor in train_dataset:

    boxes = [tensor_to_box(tensor) for tensor in boxes_tensor]

    game = BinPackingGame(tensor_to_box(container_tensor), boxes)
    dataset_keys.add(hash(game))

In [21]:
def generate_sequence(model, container, max_seq_len=10, teacher_forcing_ratio=0.0):
    """
    Genera una secuencia de cajas para un contenedor dado utilizando un modelo entrenado.
    
    Args:
        model: El modelo entrenado.
        container: Tensor con las dimensiones del contenedor (batch_size, input_dim).
        max_seq_len: Longitud máxima de la secuencia que se desea generar.
        teacher_forcing_ratio: Probabilidad de usar teacher forcing (aunque normalmente se usa 0.0 aquí).
        
    Returns:
        Secuencia generada de dimensiones (seq_len, 2), con el formato de (width, height).
    """
    model.eval()  # Poner el modelo en modo evaluación
    
    # Verifica que el contenedor sea bidimensional
    if container.ndim == 1:
        container = container.unsqueeze(0)  # Convertir a (1, input_dim)
    container = container.to(DEVICE)  # (batch_size, input_dim)
    
    # Embedding inicial del contenedor
    container_emb = model.embedding(container).unsqueeze(1)  # (batch_size, 1, hidden_dim)

    # Inicializa la secuencia generada y el estado oculto del LSTM
    generated_seq = container_emb
    hidden = None
    generated_boxes = []

    with torch.no_grad():
        for _ in range(max_seq_len):
            # Paso del LSTM
            output, hidden = model.lstm(generated_seq, hidden)  # (batch_size, seq_len, hidden_dim)
            
            # Logits para predicciones de width y height
            logits_width = model.fc_width(output[:, -1, :])  # (batch_size, max_dim+1)
            logits_height = model.fc_height(output[:, -1, :])  # (batch_size, max_dim+1)
            
            # Predicciones
            prob_width = F.softmax(logits_width, dim=-1)
            prob_height = F.softmax(logits_height, dim=-1)
            next_width = torch.multinomial(prob_width, num_samples=1).squeeze(-1)  # (batch_size,)
            next_height = torch.multinomial(prob_height, num_samples=1).squeeze(-1)  # (batch_size,)

            # Construir la siguiente caja
            next_box = torch.stack([next_width, next_height], dim=1)  # (batch_size, 2)

            #Si el ancho o el alto es 0 se termina la secuencia
            if next_box[0][0] == 0 or next_box[0][1] == 0:
                break

            generated_boxes.append(next_box.cpu().numpy())  # Guardar la predicción
            
            # Embedding de la siguiente caja
            next_box_emb = model.embedding(next_box.float().to(DEVICE)).unsqueeze(1)  # (batch_size, 1, hidden_dim)

            # Actualiza la secuencia generada agregando la nueva caja
            generated_seq = torch.cat([generated_seq, next_box_emb], dim=1)

    return generated_boxes


# Ejemplo de uso:
container = torch.tensor([[10, 10]], dtype=torch.float32)  # Ejemplo de contenedor (width=10, height=10)
print(f"{container.shape=}")
generated_seq = generate_sequence(model, container, max_seq_len=10, teacher_forcing_ratio=0.0)

# Mostrar la secuencia generada
print("Secuencia generada:")
for box in generated_seq:
    print(f"Ancho: {box[0][0]}, Alto: {box[0][1]}")


container.shape=torch.Size([1, 2])
Secuencia generada:
Ancho: 1, Alto: 3
Ancho: 10, Alto: 1
Ancho: 3, Alto: 5
Ancho: 7, Alto: 1
Ancho: 1, Alto: 6
Ancho: 1, Alto: 1
Ancho: 1, Alto: 4


In [33]:
from typing import Counter

from models import BinPackingGame, Box


# Generación de cajas
container_width = 10
container_height = 8

attempts = 50
valid_games = 0
games = []
unique_games = set()
coverages = set()
boxes_count = Counter()
new_games = 0
model.eval()
for i in range(attempts):
    container = torch.tensor([[container_width, container_height]],dtype=torch.float32).to(DEVICE)
    generated_boxes = generate_sequence(model, container, max_seq_len=10, teacher_forcing_ratio=0.0)

    # Paso 2: Convertir a lista de tuplas
    # box_list = [tensor_to_box(tensor) for tensor in output]

    boxes = [Box(int(gen_box[0][0]), int(gen_box[0][1])) for gen_box in generated_boxes]

    valid_boxes = [box for box in boxes if box.width > 0 and box.height > 0]
    if len(valid_boxes) == 0:
        continue
    game = BinPackingGame(Box(container_width, container_height), valid_boxes)
    games.append(game)
    result = game.solve()
    if isinstance(result, ResolvedBinPackingGameResult):
        valid_games += 1
        game_key = hash(game)
        boxes_count[len(game.boxes)] += 1
        # print(f'{game_key=}')
        if game_key not in unique_games:
            unique_games.add(game_key)
            coverages.add(game.coverage())
            if game_key not in dataset_keys:
                new_games += 1

print(f"Valid games: {valid_games}/{attempts}")
print(f"Unique games: {len(unique_games)}/{attempts}")
print(f"unique_games keys: {unique_games}")
print(f"Coverages: {coverages}")
print(f"New games: {new_games}")
print(f"Boxes count: {boxes_count}")
[print(f'{game}') for game in games]


Valid games: 16/50
Unique games: 16/50
unique_games keys: {1904146084869635521, 4604776546048649505, 7077992645349355590, 1204996520032874761, -3904023141944031124, 2872195147997666122, -5927144692391268660, 8521919048154952460, 4804942357064857743, 2055797294419686002, -6317749523471701483, 1918916807093795958, 579696818835013047, 7161926823480877142, -803580345184116294, 8392948918768709368}
Coverages: {0.3375, 0.825, 0.925, 0.5125, 0.775, 0.75, 0.8, 0.875, 0.6375, 0.375, 0.65, 0.9625, 0.1125, 0.6}
New games: 16
Boxes count: Counter({5: 7, 2: 3, 6: 3, 3: 2, 7: 1})
BinPackingGame(container=Box(width=10, height=8), boxes=[Box(width=4, height=5), Box(width=7, height=2), Box(width=8, height=3), Box(width=4, height=3), Box(width=4, height=4)])
BinPackingGame(container=Box(width=10, height=8), boxes=[Box(width=2, height=1), Box(width=6, height=9), Box(width=6, height=4), Box(width=2, height=6), Box(width=15, height=2), Box(width=2, height=1)])
BinPackingGame(container=Box(width=10, height=

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]